In [1]:
import csv
import pandas as pd

In [2]:
#para todos retornos de execução de códigos sejam apresentados
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Definição das colunas de acordo com o leiaute fornecido pela B3
# nomes das colunas
nm_colunas = [
    'TIPREG',
    'DTPREG',
    'CODBDI',
    'CODNEG',
    'TPMERC',
    'NOMRES',
    'ESPECI',
    'PRAZOT',
    'MODREF',
    'PREABE',
    'PREMAX',
    'PREMIN',
    'PREMED',
    'PREULT',
    'PREOFC',
    'PREOFV',
    'TOTNEG',
    'QUATOT',
    'VOLTOT',
    'PREEXE',
    'INDOPC',
    'DATVEN',
    'FATCOT',
    'PTOEXE',
    'CODISI',
    'DISMES'    
]

#tamanho das colunas
tam_colunas = [
    2,8,2,12,3,12,10,3,4,11,11,11,11,11,11,11,5,18,16,11,1,8,7,7,12,3    
]

len(tam_colunas)
len(nm_colunas)


26

26

In [12]:

 #df = pd.read_fwf('../../data/demo.txt',widths=tam_colunas, names=nm_colunas, skiprows=1, skipfooter=1)
 df = pd.read_fwf('../../data/2021.txt',widths=tam_colunas, names=nm_colunas, skiprows=1, skipfooter=1)


In [6]:
#df[df.isna().any(axis=1)] -> para selecionar as linhas contendo valores NaN  
df.isna().any() 

#somente PRAZOTOT possui NaN, não havendo necessidade de nenhum tipo de tratamento. Essa variável somente faz sentido para 
#mercado a termo, cujas observações não serão utilizadas neste modelo

TIPREG    False
DTPREG    False
CODBDI    False
CODNEG    False
TPMERC    False
NOMRES    False
ESPECI    False
PRAZOT     True
MODREF    False
PREABE    False
PREMAX    False
PREMIN    False
PREMED    False
PREULT    False
PREOFC    False
PREOFV    False
TOTNEG    False
QUATOT    False
VOLTOT    False
PREEXE    False
INDOPC    False
DATVEN    False
FATCOT    False
PTOEXE    False
CODISI    False
DISMES    False
dtype: bool

In [36]:
# o campo CODBDI indica o código dos papéis na emissão do boletim diário de informações
# apesar de iteressar somente o CODBDI=2 ou 96 (LOTE PADRAO OU FRACIONARIO), uma análise de frequência nos indica que o arquivo possui também grande volume
# de negociação de "opções"
df['CODBDI'].unique()
df.groupby('CODBDI').count()




array([ 2, 96], dtype=int64)

,TIPREG,DTPREG,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,PREMAX,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
CODBDI,,,,,,,,,,,,,,,,,,,,,
2,195327,195327,195327,195327,195327,195327,0,195327,195327,195327,...,195327,195327,195327,195327,195327,195327,195327,195327,195327,195327
96,110390,110390,110390,110390,110390,110390,0,110390,110390,110390,...,110390,110390,110390,110390,110390,110390,110390,110390,110390,110390


In [22]:
# filtros
# Critério 1 - somente papéis negociados em R$
df = df.query("MODREF=='R$'")

# Critério 2 - somente ACÕES ORDINÁRIAS e PREFERENCIAIS
# este critério possui dois objetivos:
# - selecionar somente os tipos de ativos mais negociados pelo ivenstidor comum
# - reduzir o número de registros a ser trabalhado
df = df[df.ESPECI.str.contains('ON') + df.ESPECI.str.contains('PN')]

# Critério 3 - somente LOTE PADRÃO e FRACIONÁRIO
# para este modelo, não temos interesse em outros tipos de ativo, como por exemplo recuperação judicial, opções ou leilões.
df = df.query("CODBDI in (2,96)")

# com os filtros acima aplicados, sobraram somente observações TPMERC(TIPO DE MERCADO) = 10 e 20. Ou seja, "Mercado a Vista" e "Fracionário"
df['TPMERC'].unique()
df['ESPECI'].unique()
df.count()


array([10, 20], dtype=int64)

array(['ON      NM', 'ON      N2', 'PN      N2', 'PN', 'ON      N1',
       'PN      N1', 'ON', 'PNA', 'PNB', 'ON  EJ', 'PNA     N1',
       'ON  ES  NM', 'ON  EJ  N1', 'PN  EJ  N1', 'PNB     N1',
       'ON  EDJ NM', 'ON      MA', 'ON  ED  NM', 'ON  EJ  NM', 'PN  EJ',
       'PNA     N2', 'ON  EDR NM', 'PNE', 'PNF', 'PNA EJ  N1',
       'PNB EJ  N1', 'PNA ED', 'PND', 'PN  EJ  N2', 'PN  EJS N2',
       'ON  ED', 'ON  ED  N1', 'PNB ED  N1', 'ON      MB', 'PNC',
       'ON  EJ  N2', 'PNA EJ  N2', 'ON  EB', 'PN  EB', 'PNA EJ', 'PNB EJ',
       'PN  ED', 'PN  ED  N1', 'ON  EJS N1', 'PN  EJS N1', 'ON  ATZ NM',
       'ON  REC N2', 'PNE ED', 'ON  EDJ', 'PNA ED  N1', 'PND ED',
       'ON  ES', 'ON  REC NM', 'ON  EDJ N1', 'PN  EDJ N1', 'ON  EB  NM',
       'PNA     MB', 'PNA REC N2', 'PNB ED', 'PN  REC N2', 'PNB     MB',
       'ON  ERJ NM', 'ON  EC  NM', 'ON  ED  N2', 'PN  ED  N2',
       'ON  EB  N1', 'PNB EB  N1', 'ON  ES  N2', 'PN  ES  N2', 'ON  ERJ',
       'PNB EDJ N1', 'ON  EB  N2', 'PN

TIPREG    192287
DTPREG    192287
CODBDI    192287
CODNEG    192287
TPMERC    192287
NOMRES    192287
ESPECI    192287
PRAZOT         0
MODREF    192287
PREABE    192287
PREMAX    192287
PREMIN    192287
PREMED    192287
PREULT    192287
PREOFC    192287
PREOFV    192287
TOTNEG    192287
QUATOT    192287
VOLTOT    192287
PREEXE    192287
INDOPC    192287
DATVEN    192287
FATCOT    192287
PTOEXE    192287
CODISI    192287
DISMES    192287
dtype: int64

In [86]:
#df['NOMRES'].unique()
[x for x in df['NOMRES'].unique() if x.find('PETR4')!=-1]

len(df['DTPREG'].unique())

df.sort_values(['DTPREG','CODNEG'])




[]

247

,TIPREG,DTPREG,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
140,1,20210104,2,AALR3,10,ALLIAR,ON NM,NaN,R$,11,...,50000,1113015080000,40510000,4458,3,19000000,0,9999,123100000010,0
4788,1,20210104,96,AALR3F,20,ALLIAR,ON NM,NaN,R$,11,...,0,1105004210000,811500,89,2,73210000,0,9999,123100000010,0
143,1,20210104,2,ABCB4,10,ABC BRASIL,PN N2,NaN,R$,15,...,70000,1620059860000,157050000,25320,3,26000000,0,9999,123100000010,0
4789,1,20210104,96,ABCB4F,20,ABC BRASIL,PN N2,NaN,R$,15,...,50000,1630024680000,4722400,761,3,61730000,0,9999,123100000010,0
144,1,20210104,2,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,15,...,80000,1550419370000,1783520000,276850,7,86000000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054610,1,20211230,96,WIZS3F,20,WIZ S.A.,ON NM,NaN,R$,7,...,40000,805007740000,1567800,124,1,83210000,0,9999,123100000010,0
1043265,1,20211230,2,WLMM4,10,WLM IND COM,PN,NaN,R$,36,...,20000,3985000070000,120000,45,8,56000000,0,9999,123100000010,0
1060107,1,20211230,96,WLMM4F,20,WLM IND COM,PN,NaN,R$,44,...,50000,4346000010000,100,0,0,44000000,0,9999,123100000010,0
1056670,1,20211230,2,YDUQ3,10,YDUQS PART,ON NM,NaN,R$,20,...,50000,2056085250000,205300000,42194,6,2000000,0,9999,123100000010,0


In [68]:

len(df['CODNEG'].unique())
df.sort_values(['CODNEG','DTPREG']).groupby(['CODNEG']).first()


#percebe-se que não existem valores para todos os ativos para todas as datas
#será necessário preencher as informações que estão faltando (ou talvez não)



997

,TIPREG,DTPREG,CODBDI,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,PREMAX,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
CODNEG,,,,,,,,,,,,,,,,,,,,,
AALR3,1,20210104,2,10,ALLIAR,ON NM,NaN,R$,11,13000000000,...,50000,1113015080000,40510000,4458,3,19000000,0,9999,123100000010,0
AALR3F,1,20210104,96,20,ALLIAR,ON NM,NaN,R$,11,6000000000,...,0,1105004210000,811500,89,2,73210000,0,9999,123100000010,0
ABCB10F,1,20210219,96,20,ABC BRASIL,PN REC N2,NaN,R$,14,65000000000,...,0,1480000340000,48500,7,0,47180000,0,9999,123100000010,0
ABCB4,1,20210104,2,10,ABC BRASIL,PN N2,NaN,R$,15,89000000000,...,70000,1620059860000,157050000,25320,3,26000000,0,9999,123100000010,0
ABCB4F,1,20210104,96,20,ABC BRASIL,PN N2,NaN,R$,15,80000000000,...,50000,1630024680000,4722400,761,3,61730000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM3F,1,20210104,96,20,WLM IND COM,ON,NaN,R$,22,0,...,0,2164000110000,20000,4,3,18340000,0,9999,123100000010,0
WLMM4,1,20210104,2,10,WLM IND COM,PN,NaN,R$,31,30000000000,...,0,3200000050000,80000,25,1,62000000,0,9999,123100000010,0
WLMM4F,1,20210104,96,20,WLM IND COM,PN,NaN,R$,32,8000000000,...,0,3296000090000,6800,2,1,68440000,0,9999,123100000010,0


In [77]:
# importar os pacotes necessários
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt

In [92]:
#pegando somente as ações da PETROBRAS para iniciar
df_pet = df.query("CODNEG=='PETR4'")
df_pet = df_pet.sort_values(['DTPREG','CODNEG'])


,TIPREG,DTPREG,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
2749,1,20210104,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,28,...,0,2891093230000,7471970000,2153768,7,58000000,0,9999,123100000010,0
8616,1,20210105,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,28,...,30000,3004073890000,9518110000,2794814,2,48000000,0,9999,123100000010,0
14477,1,20210106,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,30,...,60000,3007101070000,9656250000,2947101,5,88000000,0,9999,123100000010,0
20438,1,20210107,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,30,...,0,3101755790000,5617130000,1730397,1,79000000,0,9999,123100000010,0
26659,1,20210108,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,31,...,0,3112829010000,6713630000,2084999,7,58000000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996357,1,20211223,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,28,...,20000,2833433810000,2871360000,812335,5,62000000,0,9999,123100000010,0
996358,1,20211227,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,28,...,50000,2876460600000,4422790000,1265090,3,1000000,0,9999,123100000010,0
996359,1,20211228,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,29,...,70000,2878431420000,3068810000,886001,6,99000000,0,9999,123100000010,0
996360,1,20211229,2,PETR4,10,PETROBRAS,PN N2,NaN,R$,28,...,30000,2854610880000,3550840000,1015407,3,91000000,0,9999,123100000010,0


In [90]:
#dividir a base para treinar o modelo. 75% para treino e 25% para teste
#df.sort_values(['DTPREG','CODNEG'])
#(train, test) = train_test_split(df)
#train
#test





,TIPREG,DTPREG,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
140,1,20210104,2,AALR3,10,ALLIAR,ON NM,NaN,R$,11,...,50000,1113015080000,40510000,4458,3,19000000,0,9999,123100000010,0
4788,1,20210104,96,AALR3F,20,ALLIAR,ON NM,NaN,R$,11,...,0,1105004210000,811500,89,2,73210000,0,9999,123100000010,0
143,1,20210104,2,ABCB4,10,ABC BRASIL,PN N2,NaN,R$,15,...,70000,1620059860000,157050000,25320,3,26000000,0,9999,123100000010,0
4789,1,20210104,96,ABCB4F,20,ABC BRASIL,PN N2,NaN,R$,15,...,50000,1630024680000,4722400,761,3,61730000,0,9999,123100000010,0
144,1,20210104,2,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,15,...,80000,1550419370000,1783520000,276850,7,86000000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054610,1,20211230,96,WIZS3F,20,WIZ S.A.,ON NM,NaN,R$,7,...,40000,805007740000,1567800,124,1,83210000,0,9999,123100000010,0
1043265,1,20211230,2,WLMM4,10,WLM IND COM,PN,NaN,R$,36,...,20000,3985000070000,120000,45,8,56000000,0,9999,123100000010,0
1060107,1,20211230,96,WLMM4F,20,WLM IND COM,PN,NaN,R$,44,...,50000,4346000010000,100,0,0,44000000,0,9999,123100000010,0
1056670,1,20211230,2,YDUQ3,10,YDUQS PART,ON NM,NaN,R$,20,...,50000,2056085250000,205300000,42194,6,2000000,0,9999,123100000010,0


,TIPREG,DTPREG,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
73475,1,20210305,2,MDIA3,10,M.DIASBRANCO,ON NM,NaN,R$,30,...,90000,2981052730000,88470000,26301,1,54000000,0,9999,123100000010,0
914396,1,20211014,96,CRPG5F,20,CRISTAL,PNA,NaN,R$,99,...,40000,9999001280000,121000,119,7,26480000,0,9999,123100000010,0
178140,1,20210623,96,BRGE5F,20,ALFA CONSORC,PNA,NaN,R$,10,...,30000,1198000010000,500,0,0,51100000,0,9999,123100000010,0
891839,1,20210903,96,BOBR4F,20,BOMBRIL,PN,NaN,R$,2,...,40000,210001480000,133300,2,7,38330000,0,9999,123100000010,0
902666,1,20210810,2,CAML3,10,CAMIL,ON NM,NaN,R$,8,...,60000,887044140000,127830000,11359,2,13000000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046456,1,20210708,2,USIM5,10,USIMINAS,PNA N1,NaN,R$,19,...,70000,1958265950000,1810930000,350792,1,94000000,0,9999,123100000010,0
134643,1,20210512,96,WHRL4F,20,WHIRLPOOL,PN,NaN,R$,7,...,10000,789001230000,146700,11,4,61560000,0,9999,123100000010,0
175891,1,20210622,2,MGLU3,10,MAGAZ LUIZA,ON NM,NaN,R$,21,...,70000,2099499390000,1639920000,342756,2,49000000,0,9999,123100000010,0
1004218,1,20211007,2,RADL3,10,RAIADROGASIL,ON NM,NaN,R$,22,...,30000,2307201860000,1230730000,282517,5,25000000,0,9999,123100000010,0


,TIPREG,DTPREG,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
1041835,1,20211206,96,TELB4F,20,TELEBRAS,PN,NaN,R$,16,...,70000,1699000040000,6800,1,1,50050000,0,9999,123100000010,0
1040853,1,20210823,96,TRPL3F,20,TRAN PAULIST,ON N1,NaN,R$,31,...,20000,3196000800000,68100,21,4,37340000,0,9999,123100000010,0
42759,1,20210126,96,BMIN3F,20,MERC INVEST,ON,NaN,R$,19,...,10000,2099000010000,1400,0,2,73280000,0,9999,123100000010,0
1003968,1,20211221,2,RAIZ4,10,RAIZEN,PN N2,NaN,R$,5,...,50000,576088520000,504980000,28938,6,73000000,0,9999,123100000010,0
41110,1,20210208,96,TESA3F,20,TERRA SANTA,ON NM,NaN,R$,27,...,40000,2840000750000,142700,39,8,62200000,0,9999,123100000010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91696,1,20210310,2,RADL3,10,RAIADROGASIL,ON NM,NaN,R$,23,...,50000,2437469840000,1295540000,311398,8,45000000,0,9999,123100000010,0
904894,1,20211221,2,CEBR3,10,CEB,ON,NaN,R$,16,...,20000,1600000070000,200000,32,2,14000000,0,9999,123100000010,0
1039501,1,20211228,2,AMER3,10,AMERICANAS,ON NM,NaN,R$,32,...,30000,3104119530000,306940000,96055,4,85000000,0,9999,123100000010,0
17661,1,20210218,96,WHRL3F,20,WHIRLPOOL,ON,NaN,R$,7,...,50000,736000530000,47500,3,4,81440000,0,9999,123100000010,0


In [104]:
#número de dias utilizados para predição (input)
dias_input = 30

In [96]:
type(df_pet['PREULT'])

valores = df_pet['PREULT']
valores

pandas.core.series.Series

2749      2882000
8616      2936000
14477     3052000
20438     3080000
26659     3105000
           ...   
996357    2829000
996358    2860000
996359    2887000
996360    2859000
996361    2851000
Name: PREULT, Length: 247, dtype: int64

In [188]:
#realiza a normalização dos dados (z-score)
import scipy.stats as stats
type (valores)
valores_z=stats.zscore(np.array(valores))
type (valores_z)

pandas.core.series.Series

numpy.ndarray

In [185]:
#dividindo em treino, teste e validação

tam_treino = int(len(valores_z) * 0.70)
tam_teste = len(valores_z) - tam_treino
tam_treino
tam_teste
type(valores_z[i-dias_input:i])

172

75

numpy.ndarray

In [269]:
#gerando a base de treino
input_treino, saida_treino = [], []
for i in range(dias_input, tam_treino):
    input_treino.append(valores_z[i-dias_input:i])
    saida_treino.append(valores_z[i])

    
input_treino = np.array(input_treino)
saida_treino = np.array(saida_treino)
saida_treino
len(input_treino)
len(saida_treino)

array([[0.87586347, 1.11860178, 1.64003965, 1.76590396, 1.87828281,
        1.74342819, 1.86030219, 1.22648548, 1.07365024, 0.66459123,
        0.66459123, 0.65110577, 0.73651369, 0.41735776, 0.02627936,
        0.21957098, 0.25103706, 0.45331899, 0.11618244, 0.1791146 ,
        0.81293131, 0.89833924, 0.78596039, 1.11410663, 0.67807669,
        0.38139653, 0.32295953, 0.61064938, 0.59266877, 0.94329078]])

array([ 1.33886433e+00,  3.72406220e-01, -2.17634610e+00, -1.52904392e+00,
       -1.08402368e+00, -1.20988799e+00, -1.97406417e+00, -1.99653994e+00,
       -2.39211349e+00, -2.60338573e+00, -2.19432671e+00, -1.92911263e+00,
       -2.34266679e+00, -2.41908441e+00, -2.17185094e+00, -1.70435493e+00,
       -1.70885008e+00, -1.56950031e+00, -1.55601484e+00, -1.37171353e+00,
       -1.49308269e+00, -1.38070384e+00, -1.52454877e+00, -1.65490823e+00,
       -1.65940339e+00, -1.86618047e+00, -1.50656815e+00, -1.47060692e+00,
       -1.37171353e+00, -1.24135407e+00, -1.28630561e+00, -1.33125714e+00,
       -1.24584922e+00, -1.23685891e+00, -1.43464569e+00, -1.45262630e+00,
       -1.30428622e+00, -1.31777168e+00, -1.13347037e+00, -1.55151969e+00,
       -1.74481131e+00, -1.24584922e+00, -1.28181045e+00, -1.34923776e+00,
       -1.42565538e+00, -1.38070384e+00, -1.50207300e+00, -1.39868445e+00,
       -1.43015053e+00, -1.43464569e+00, -1.52005361e+00, -1.65940339e+00,
       -1.41666507e+00, -

142

142

In [224]:
#treinando o modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(dias_input, 1)))
#model.add(Dropout(0.3))
model.add(LSTM(100, return_sequences=False, input_shape=(dias_input, 1)))
#model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

model.summary()

result = model.fit(input_treino, saida_treino, epochs=30, batch_size=30)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 30, 100)           40800     
                                                                 
 lstm_29 (LSTM)              (None, 100)               80400     
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
5/5 [==============================] - 8s 59ms/step - loss: 0.8026
Epoch 2/30
5/5 [==============================] - 0s 56ms/step - loss: 0.5269
Epoch 3/30
5/5 [==============================] - 0s 64ms/step - loss: 0.3860
Epoch 4/30
5/5 [==============================] - 0s 56ms/step - loss: 0.3181

In [262]:
#testando o modelo
#pega os ultimos 30 dias antes do inicio dos testes
input_teste = []

input_teste.append(valores_z[tam_treino-dias_input:tam_treino])
teste = np.array(valores_z[tam_treino-dias_input:tam_treino])
input_teste = np.array(input_teste)
teste = np.array(teste)



In [263]:

input_teste
predicted = []

for i in (range(tam_treino, tam_treino + tam_teste)):
    result=model.predict(input_teste,batch_size=30)
    result[0,0]
    predicted = predicted.append(result[0,0])
    predicted
    input_teste = np.delete(input_teste,0)
    np.array(input_teste)
    input_teste = [np.append(input_teste,result[0,0])]
    input_teste


#type(input_teste)

#input_teste

#result = model.predict(input_teste)
#result


array([[ 0.17012429, -0.00518672, -0.20297349, -0.26590565,  0.75898946,
         0.64661061,  0.51625115,  0.74100885,  0.76348462,  0.92980532,
         1.10961147,  0.82192162,  0.05774544,  0.04875513, -0.16701226,
        -0.20746865,  0.10719213,  0.25103706,  0.33644499,  0.3454353 ,
         0.56569784,  0.70954277,  0.2959886 ,  0.01728905,  0.00380359,
        -0.24342988, -0.16251711, -0.61203251, -0.92219813, -0.57607127]])

1/1 [==============================] - 0s 37ms/step


-0.6353628

array([-0.00518672, -0.20297349, -0.26590565,  0.75898946,  0.64661061,
        0.51625115,  0.74100885,  0.76348462,  0.92980532,  1.10961147,
        0.82192162,  0.05774544,  0.04875513, -0.16701226, -0.20746865,
        0.10719213,  0.25103706,  0.33644499,  0.3454353 ,  0.56569784,
        0.70954277,  0.2959886 ,  0.01728905,  0.00380359, -0.24342988,
       -0.16251711, -0.61203251, -0.92219813, -0.57607127])

[array([-0.00518672, -0.20297349, -0.26590565,  0.75898946,  0.64661061,
         0.51625115,  0.74100885,  0.76348462,  0.92980532,  1.10961147,
         0.82192162,  0.05774544,  0.04875513, -0.16701226, -0.20746865,
         0.10719213,  0.25103706,  0.33644499,  0.3454353 ,  0.56569784,
         0.70954277,  0.2959886 ,  0.01728905,  0.00380359, -0.24342988,
        -0.16251711, -0.61203251, -0.92219813, -0.57607127, -0.6353628 ])]

ValueError: in user code:

    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Júlio\AppData\Roaming\Python\Python39\site-packages\keras\e

In [230]:
predicted


[array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dtype=float32),
 array([[-0.6353628]], dt

In [139]:

X_test = []
for i in range(dias_input, dias_input + tam_teste):
    X_test.append(valores_z[i-dias_input:i])
    
X_test[0:1]
input_teste=[]
input_teste.append(np.array(valores_z[tam_treino-dias_input:tam_treino]))
input_teste
result=model.predict()

AttributeError: 'Sequential' object has no attribute 'shape'

In [ ]:
stats.zscore()

In [ ]:
for i in (range(tam_traino, tam_teste)):
    result=model.predict(input_teste,batch_size=30)
    input_teste.append(result)
    input_teste.drop(0)